In [1]:
!nvidia-smi

Thu Jun  9 13:58:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   39C    P0    26W /  70W |  15094MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Settings

In [2]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

## Image Pre-Process

In [6]:
train_png = sorted(glob('/home/lab17/jupyter_home/Data/product_image/Training/image/**/*.jpg'))
test_png = sorted(glob('/home/lab17/jupyter_home/Data/product_image/Validation/image/**/*.jpg'))

# files = os.listdir('/home/lab17/jupyter_home/Data/bdata_snacks')
# files

In [7]:
print(len(train_png), len(set(train_png)))
print(len(test_png), len(set(test_png)))

8664 8664
1140 1140


In [10]:
# # decoder
# files = os.listdir('/home/lab17/jupyter_home/Data/bdata_snacks')
# train_labels = sorted(files)
# train_dict = {i:l[:-4] for i, l in zip(range(311), train_labels)}

In [8]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (256, 256),interpolation = cv2.INTER_AREA)
    return img

In [9]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|███████████████████████████████████████| 1140/1140 [04:06<00:00,  4.62it/s]


In [11]:
np.save('/home/lab17/jupyter_home/Data/0609_product_train.npy', np.array(train_imgs))
np.save('/home/lab17/jupyter_home/Data/0609_product_valid.npy', np.array(test_imgs))

In [12]:
train_imgs = np.load('/home/lab17/jupyter_home/Data/0609_product_train.npy')
test_imgs = np.load('/home/lab17/jupyter_home/Data/0609_product_valid.npy')

In [13]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",meanR, meanG, meanB)
print("train 표준편차",stdR, stdG, stdB)

train 평균 0.7320190225460682 0.7214040975750421 0.698442921707468
train 표준편차 0.1102257821729469 0.13662054597674614 0.18693838057330145


In [14]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",meanR, meanG, meanB)
print("test 표준편차",stdR, stdG, stdB)

test 평균 0.7334272809154929 0.7224970039863705 0.7002222745755632
test 표준편차 0.10891077610936534 0.13595300945229055 0.18671652429907093
